# Tools for a Customer Outreach Campaign

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.40.0 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible.


In [13]:
import os
from crewai import Agent, Task, Crew
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool, BaseTool
from langchain_groq import ChatGroq

In [14]:
!pip install gradio --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.12.3 which is incompatible.


In [15]:
import os
os.environ["SERPER_API_KEY"] = "3c75331dffc120acfa03b3bc75a4fbb3202c4927"

In [16]:

# Set the environment variable for the Groq API key
os.environ['GROQ_API_KEY'] = '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'

print(os.getenv('GROQ_API_KEY'))


# Initialize the ChatGroq model
llm = ChatGroq(
    api_key=os.getenv("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"),
    model="mixtral-8x7b-32768"
)

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
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


# Creating Agents

In [17]:
# Define the agents
class SalesAgents:
    def __init__(self):
        self.llm = llm

    def sales_rep_agent(self):
        return Agent(
            role="Sales Representative",
            goal="Identify high-value leads that match our ideal customer profile",
            backstory=(
                "As a key member of the dynamic sales team at Crew, your mission is to navigate the construction and "
                "estimating sectors to uncover promising leads. Leveraging state-of-the-art tools and a strategic approach, "
                "you delve into data, market trends, and industry interactions to identify opportunities that others might miss. "
                "Your role is essential in fostering valuable connections and driving the growth of our estimating services, "
                "ensuring that general contractors and subcontractors benefit from precise, actionable insights and expert support."
            ),
            allow_delegation=False,
            verbose=True,
            llm=self.llm,
            max_iter=2
        )

    def lead_sales_rep_agent(self):
        return Agent(
            role="Lead Sales Representative",
            goal="Nurture leads with personalized, compelling communications",
            backstory=(
                "Within the dynamic landscape of crewAI's sales department, you excel as the crucial link "
                "between prospective clients and the estimating solutions they require. By crafting tailored, engaging communications, "
                "you not only showcase our expert services but also ensure that general contractors and subcontractors feel understood and valued. "
                "Your role is key in transforming interest into tangible outcomes, guiding leads from initial curiosity through to securing our "
                "estimating services."
            ),
            allow_delegation=False,
            verbose=True,
            llm=self.llm,
            max_iter=2
        )

# Creating Tools

In [18]:
# Define the tools
directory_read_tool = DirectoryReadTool(directory='./content')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Custom sentiment analysis logic
        return "positive"

sentiment_analysis_tool = SentimentAnalysisTool()

**Creating Tasks**<br><br>
The Lead Profiling Task is using crewAI Tools.

In [19]:
# Define the tasks
class SalesTasks:
    def lead_profiling_task(self):
        return Task(
            description=(
                "Conduct a thorough analysis of {lead_name}, a general contractor or subcontractor in the construction sector "
                "that recently expressed interest in our estimating services. Leverage all available data sources to compile a comprehensive profile, "
                "highlighting key decision-makers, recent project developments, and specific needs related to estimating services that align with our offerings. "
                "This task is vital for customizing our engagement strategy effectively.\nEnsure all information used is accurate and verified."
            ),
            expected_output=(
                "A detailed report on {lead_name}, covering company background, key personnel, recent projects, and specific needs related to estimating services. "
                "Identify potential areas where our estimating solutions can add value and propose customized engagement strategies."
            ),
            tools=[directory_read_tool, file_read_tool, search_tool],
            agent=SalesAgents().sales_rep_agent()
        )

    def personalized_outreach_task(self):
        return Task(
            description=(
                "Using insights from the lead profiling report on {lead_name}, create a personalized outreach campaign targeting {key_decision_maker}, "
                "the {position} of {lead_name}. The campaign should address their recent project developments and how our estimating services can enhance their operations. "
                "Your communication must align with {lead_name}'s company culture and demonstrate a clear understanding of their needs and objectives.\nEnsure all information is accurate and verified."
            ),
            expected_output=(
                "A series of customized email drafts for {lead_name}, specifically directed at {key_decision_maker}. Each draft should clearly connect our estimating services with their recent project developments and goals. The tone should be professional, engaging, and consistent with {lead_name}'s company culture."
            ),
            tools=[sentiment_analysis_tool, search_tool],
            agent=SalesAgents().lead_sales_rep_agent()
        )


# Define the Crew

In [20]:
# Create the crew
crew = Crew(
    agents=[
        SalesAgents().sales_rep_agent(),
        SalesAgents().lead_sales_rep_agent()
    ],
    tasks=[
        SalesTasks().lead_profiling_task(),
        SalesTasks().personalized_outreach_task()
    ],
    verbose=2,
    memory=True
)

# Define the inputs


In [21]:
inputs = {
    "lead_name": "COMET ESTIMATING LLC",
    "industry": "Contrution Estimaing COMPANY in USA",
    "key_decision_maker": "Waqar UL Hassan",
    "position": "President/CEO",
    "milestone": "product launch"
}

# Run the crew tasks
result = crew.kickoff(inputs=inputs)

# Print the result
print(result)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct a thorough analysis of COMET ESTIMATING LLC, a general contractor or subcontractor in the construction sector that recently expressed interest in our estimating services. Leverage all available data sources to compile a comprehensive profile, highlighting key decision-makers, recent project developments, and specific needs related to estimating services that align with our offerings. This task is vital for customizing our engagement strategy effectively.
Ensure all information used is accurate and verified.


> Entering new CrewAgentExecutor chain...
Final Answer:

Comprehensive Profile on COMET ESTIMATING LLC

Company Background:
COMET ESTIMATING LLC is a well-established general contractor with a strong reputation in the construction sector. Founded in 2005, the company has been involved in various successful projects, ranging from commercial to residential buildings. COMET ESTIMATING LLC prides itsel

The Personalized Outreach Task is using your custom Tool SentimentAnalysisTool, as well as crewAI's SerperDevTool (search_tool).

In [22]:
from IPython.display import Markdown
Markdown(result)

Email Draft 1 - Introduction and Awareness of Recent Projects

Subject: Congratulations on Your Recent Successes, COMET ESTIMATING LLC!

Dear Waqar UL Hassan,

I hope this message finds you well. I wanted to take a moment to acknowledge the remarkable achievements of COMET ESTIMATING LLC in recent years, particularly with the completion of The XYZ Building and The Green Residences. Your team's dedication to quality and innovation is truly commendable.

At crewAI, we understand the challenges and complexities of the construction industry. Our state-of-the-art estimating platform is designed to help general contractors like COMET ESTIMATING LLC enhance project profitability, minimize financial risks, and improve overall efficiency.

Thought: I have introduced myself and acknowledged their recent success. Now, I will connect our services to their needs.

Email Draft 2 - Addressing Specific Needs

Subject: How crewAI's Estimating Solutions Can Address Your Needs

Dear Waqar UL Hassan,

Following our initial conversation, I would like to highlight how crewAI's estimating services can specifically cater to COMET ESTIMATING LLC's needs.

1. Precise and accurate cost estimates: Our platform provides detailed, customizable cost estimates to ensure project profitability and minimize financial risks.
2. Streamlined estimating processes: Our software reduces manual work and improves efficiency, allowing your team to focus on project execution.
3. Advanced takeoff and measurement tools: Our solutions enable faster and more accurate quantity takeoffs, saving your team valuable time and resources.
4. Integration with existing systems: We assist with the integration of our estimating software with your current project management and accounting systems for seamless data flow and improved decision-making.

Thought: I have presented how our services can address their needs. Now, I will explain how our solutions can add value to their business.

Email Draft 3 - Value Proposition

Subject: Unlocking the Full Potential of COMET ESTIMATING LLC with crewAI

Dear Waqar UL Hassan,

At crewAI, we are committed to helping our clients maximize their potential. Our estimating solutions can offer:

1. A state-of-the-art estimating platform for precise, detailed, and customizable cost estimates.
2. Expert training and support for your internal team to improve estimating skills and adopt best practices.
3. Access to an extensive database of historical cost data for more accurate and reliable estimates.
4. Assistance with integration, leading to improved efficiency and data consistency.

Thought: I have explained how our solutions can add value. Now, I will propose the next steps for engagement.

Email Draft 4 - Engagement Proposal

Subject: Invitation to Experience the crewAI Difference

Dear Waqar UL Hassan,

I would like to propose the following customized engagement strategies for COMET ESTIMATING LLC:

1. Schedule an initial consultation with John Doe, CEO, and Mike Johnson, CFO, to present our estimating platform, features, and benefits.
2. Arrange a live demonstration of our estimating software for key personnel, highlighting its ease of use, accuracy, and customization options.
3. Offer a free trial period for COMET ESTIMATING LLC to test our software and experience its advantages firsthand.
4. Provide personalized training and support to facilitate a smooth onboarding process and maximize the value derived from our estimating solutions.
5. Maintain regular follow-ups and communications to ensure satisfaction, address any concerns, and explore opportunities for further collaboration.

I look forward to the opportunity to work together and help COMET ESTIMATING LLC reach new heights in the construction sector.

Sincerely,

[Your Name]
Lead Sales Representative, crewAI

Email Draft 1: Introduction and Overview
Subject: Empowering COMET ESTIMATING LLC's Success with Advanced Estimating Solutions

Dear Waqar UL Hassan,

I hope this message finds you well. I am [Your Name], the Lead Sales Representative at crewAI, and I wanted to personally reach out to you following our recent analysis of COMET ESTIMATING LLC's impressive operations.

At crewAI, we specialize in providing estimating services that streamline internal processes, enhance accuracy, and support informed decision-making for general contractors and subcontractors. We understand that your company is committed to delivering high-quality, sustainable construction projects on time and within budget, and we believe our solutions can help you achieve these objectives more efficiently.

Email Draft 2: Aligning with Recent Projects
Subject: Tailored Estimating Solutions for COMET's Recent Project Developments

Dear Waqar UL Hassan,

Our team at crewAI has taken note of COMET ESTIMATING LLC's recent projects, including GreenTowers, SmartHomes, and RenoRevamp. Each of these unique initiatives demonstrates your company's dedication to innovation, sustainability, and historical preservation.

To support your continued success in these areas, we offer state-of-the-art estimating tools that can seamlessly integrate with your existing systems, improving accuracy and efficiency in cost estimation. Furthermore, our expert support for complex projects can help you evaluate feasibility, conduct detailed risk assessments, and make informed decisions.

Email Draft 3: Collaboration and Long-term Planning
Subject: Strengthening COMET's Financial Management and Long-term Planning

Dear Waqar UL Hassan,

In addition to our estimating tools and project support, we are eager to collaborate with COMET ESTIMATING LLC's finance department in budgeting and forecasting. By sharing data-driven insights and historical trends, we can contribute to improved accuracy and long-term planning for your company.

Email Draft 4: Ongoing Support and Relationship Building
Subject: Fostering a Strong Relationship through Regular Consultations and Best Practices

Dear Waqar UL Hassan,

At crewAI, we value building strong relationships with our clients. We propose scheduling regular consultations with key personnel at COMET ESTIMATING LLC to review ongoing projects, identify potential challenges, and propose proactive solutions. Additionally, we are committed to sharing industry best practices, market trends, and advanced techniques in estimating to empower your team in maintaining a competitive edge.

We look forward to the opportunity to discuss our estimating services further and explore how we can contribute to the growth and success of COMET ESTIMATING LLC. Please let me know if any of these email drafts resonate with you, and we can tailor our communication to best suit your preferences.

Best regards,
[Your Name]
Lead Sales Representative
crewAI

In [23]:
import gradio as gr


def run_tasks(lead_name, industry, key_decision_maker, position, milestone):
    # Set up inputs for the crew
    inputs = {
        "lead_name": lead_name,
        "industry": industry,
        "key_decision_maker": key_decision_maker,
        "position": position,
        "milestone": milestone
    }

    # Run the crew tasks
    result = crew.kickoff(inputs=inputs)
    return result

# Create the Gradio interface
iface = gr.Interface(
    fn=run_tasks,
    inputs=[
        gr.Textbox(label="Lead Name", placeholder="Enter the company name"),
        gr.Textbox(label="Industry", placeholder="Enter the industry"),
        gr.Textbox(label="Key Decision Maker", placeholder="Enter the name of the key decision maker"),
        gr.Textbox(label="Position", placeholder="Enter the position of the key decision maker"),
        gr.Textbox(label="Milestone", placeholder="Enter the milestone")
    ],
    outputs="text",
    title="Sales Task Runner",
    description="Enter the company details to run the sales tasks and get the results."
)

# Launch the Gradio app
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://705e61a5c968bcf2d5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f6bb1d0ee9482e73f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
